# Problem 2. Off Policy Evaluation And Causal Inference

In class we have discussed Markov decision processes (MDPs), methods for learning MDPs from data, and ways to compute optimal policies from that MDP. However, before we use that policy, we often want to get an estimate of the its performance. In some settings such as games or simulations, you are able to directly implement that policy and directly measure the performance, but in many situations such as health care implementing and evaluating a policy is very expensive and time consuming.

Thus we need methods for evaluating policies without actually implementing them. This task is usually referred to as off-policy evaluation or causal inference. In this problem we will explore different ways of estimating off policy performance and prove some of the properties of those estimators.

Most of the methods we discuss apply to general MDPs, but for the sake of this problem, we will consider MDPs with a single timestep. We consider a universe consisting of states $S$, actions $A$, a reward function $R(s,a)$ where $s$ is a state and $a$ is an action. One important factor is that we often only have a subset of $A$ in our dataset. For example, each state $s$ could represent a patient, each action $a$ could represent which drug we prescribe to that patient and $R(s,a)$ be their lifespan after prescribing that drug.

A policy is defined by a function $\pi_i(s,a)=p(a|s,\pi_i)$. In other words, $\pi_i(s,a)$ is the conditional probability of an action given a certain state and a policy.

We are given an observational dataset consisting of $(s,a,R(s,a))$ tuples.

Let $p(s)$ denote the probability density function for the distribution of state $s$ values within that dataset. Let $\pi_0(s,a)=p(a|s)$ within our observational data. $\pi_0$ corresponds to the baseline policy present in our observational data. Going back to the patient example, $p(s)$ would be the probability of seeing a particular patient s and $\pi_0(s,a)$ would be the probability of a patient receiving a drug in the observational data.

We are also given a target policy $\pi_1(s,a)$ which gives the conditional probability $p(a|s)$ in our optimal policy that we are hoping to evaluate. One particular note is that even though this is a distribution, many of the policies that we hope to evaluate are deterministic such that given a particular state $s_i$, $p(a|s_i)=1$ for a single action and $p(a|s_i)=0$ for the other actions.

Our goal is to compute the expected value of $R(s,a)$ in the same population as our observational data, but with a policy of $\pi_1$ instead of $\pi_0$. In other words, we are trying to compute:



\begin{align*}
\mathbb{E}_{s\sim p(s), a \sim \pi_1(s,a)} R(s,a).
\end{align*}

## Important Note About Notation And Simplifying Assumptions:

We haven’t really covered expected values over multiple variables such as $\mathbb{E}_{s\sim p(s), a \sim \pi_1(s,a)} R(s,a)$ in class yet. For the purpose of this question, you may make the simplifying assumption that our states and actions are discrete distributions. This expected value over multiple variables simply indicates that we are taking the expected value over the joint pair $(s,a)$ where $s$ comes from $p(s)$ and $a$ comes from $\pi_1(s,a)$. In other words, you have a $p(s,a)$ term which is the probabilities of observing that pair and we can factorize that probability to $p(s)p(a|s) = p(s)\pi_1(s,a)$. In math notation, this can be written as:

\begin{align*}
\mathbb{E}_{s\sim p(s), a \sim \pi_1(s,a)} R(s,a)
& = \sum_{(s,a)} R(s,a)p(s,a)\\
& = \sum_{(s,a)} R(s,a)p(s)p(a|s)\\
& = \sum_{(s,a)} R(s,a)p(s)\pi_1(s,a).
\end{align*}

Unfortunately, we cannot estimate this directly as we only have samples created under policy $\pi_0$ and not $\pi_1$. For this problem, we will be looking at formulas that approximate this value using expectations under $\pi_0$ that we can actually estimate.

We will make one additional assumption that each action has a non-zero probability in the observed policy $\pi_0(s,a)$. In other words, for all actions $a$ and states $s$, $\pi_0(s,a)>0$.

__Regression:__ The simplest possible estimator is to directly use our learned MDP parameters to estimate our goal. This is usually called the regression estimator. While training our MDP, we learn an estimator $\hat{R}(s,a)$ that estimates $R(s,a)$. We can now directly estimate


\begin{align*}
\mathbb{E}_{s\sim p(s), a \sim \pi_1(s,a)} R(s,a)
\end{align*}

with 


\begin{align*}
\mathbb{E}_{s\sim p(s), a \sim \pi_1(s,a)} \hat{R}(s,a).
\end{align*}

If $\hat{R}(s,a)=R(s,a)$, then this estimator is trivially correct.

We will now consider alternative approaches and explore why you might use one estimator over
another.


## (a) [2 points] Importance Sampling: 

One commonly used estimator is known as the importance sampling estimator. Let $\hat{\pi}_0$ be an estimate of the true $\pi_0$. The importance sampling estimator uses that $\hat{\pi}_0$ and has the form:


\begin{align*}
\mathbb{E}_{s\sim p(s), a \sim \pi_0(s,a)} \frac{\pi_1(s,a)}{\hat{\pi}_0(s,a)}R(s,a).
\end{align*}

Please show that if $\hat{\pi}_0 = \pi_0$, then the importance sampling estimator is equal to: 


\begin{align*}
\mathbb{E}_{s\sim p(s), a \sim \pi_1(s,a)} R(s,a).
\end{align*}

Note that this estimator only requires us to model $\pi_0$ as we have the $R(s,a)$ values for the items in the observational data. 

### Answer:

If $\hat{\pi}_0 = \pi_0$, then


\begin{align*}
\mathbb{E}_{s\sim p(s), a \sim \pi_0(s,a)} \frac{\pi_1(s,a)}{\hat{\pi}_0(s,a)}R(s,a)
& = \sum_{(s,a)} \frac{\pi_1(s,a)}{\hat{\pi}_0(s,a)}R(s,a) p(s)\pi_0(s,a)\\
& = \sum_{(s,a)} p(s)\pi_1(s,a)R(s,a)\\
& = \mathbb{E}_{s\sim p(s), a \sim \pi_1(s,a)} R(s,a).
\end{align*}

## (b) [2 points] Weighted Importance Sampling: 

One variant of the importance sampling estimator is known as the weighted importance sampling estimator. The weighted importance sampling estimator has the form:

\begin{align*}
\frac{\mathbb{E}_{s\sim p(s), a \sim \pi_0(s,a)} \frac{\pi_1(s,a)}{\hat{\pi}_0(s,a)}R(s,a)}{\mathbb{E}_{s\sim p(s), a \sim \pi_0(s,a)} \frac{\pi_1(s,a)}{\hat{\pi}_0(s,a)}}.
\end{align*}

Please show that if $\hat{\pi}_0 = \pi_0$, then the weighted importance sampling estimator is equal to:

\begin{align*}
\mathbb{E}_{s\sim p(s), a \sim \pi_1(s,a)} R(s,a).
\end{align*}

If $\hat{\pi}_0 = \pi_0$, then, by Part (a) and using the fact that 

\begin{align*}
\mathbb{E}_{s\sim p(s), a \sim \pi_0(s,a)} \frac{\pi_1(s,a)}{\hat{\pi}_0(s,a)}  
& = \sum_{(s,a)} p(s)\pi_0(s,a) \frac{\pi_1(s,a)}{\hat{\pi}_0(s,a)}\\
& = \sum_{(s,a)} p(s)\pi_1(s,a)\\
& = 1, 
\end{align*}

we have 

\begin{align*}
\frac{\mathbb{E}_{s\sim p(s), a \sim \pi_0(s,a)} \frac{\pi_1(s,a)}{\hat{\pi}_0(s,a)}R(s,a)}{\mathbb{E}_{s\sim p(s), a \sim \pi_0(s,a)} \frac{\pi_1(s,a)}{\hat{\pi}_0(s,a)}}
& = \frac{\mathbb{E}_{s\sim p(s), a \sim \pi_1(s,a)} R(s,a)}{1}\\
& = \mathbb{E}_{s\sim p(s), a \sim \pi_1(s,a)} R(s,a).
\end{align*}

## (c) [2 points] 
One issue with the weighted importance sampling estimator is that it can be biased in many finite sample situations. In finite samples, we replace the expected value with a sum over the seen values in our observational dataset. Please show that the weighted importance sampling estimator is biased in these situations.

__Hint:__ Consider the case where there is only a single data element in your observational dataset. 

### Answer:

If we have only one data element $(s_0,a_0,R(s_0,a_0))$, then the estimation given by the weighted importance sampling estimator equals

\begin{align*}
\frac{\sum_{(s,a) \in Data} \frac{\pi_1(s,a)}{\hat{\pi}_0(s,a)}R(s,a)}{\sum_{(s,a) \in Data} \frac{\pi_1(s,a)}{\hat{\pi}_0(s,a)}}
& = \frac{ \frac{\pi_1(s_0,a_0)}{\pi_0(s_0,a_0)} R(s_0,a_0)}{\frac{\pi_1(s_0,a_0)}{\pi_0(s_0,a_0)}}\\
& = R(s_0,a_0).
\end{align*}


Since $(s_0,a_0)$ is generated according to the sampling distribution $p(s)\pi_0(s,a)$, 
in this case, the expected value of the weighted importance sampling estimator is 

\begin{align*}
\mathbb{E}_{s\sim p(s), a \sim \pi_0(s,a)} R(s,a)
\end{align*}

which is not generaly equal to 

\begin{align*}
\mathbb{E}_{s\sim p(s), a \sim \pi_1(s,a)} R(s,a).
\end{align*}

## (d) [7 points] Doubly Robust: 

One final commonly used estimator is the doubly robust estimator. The doubly robust estimator has the form:

\begin{align*}
\mathbb{E}_{s\sim p(s), a \sim \pi_0(s,a)} \left( \left[\mathbb{E}_{a\sim\pi_1(s,a)}\hat{R}(s,a)\right] + \frac{\pi_1(s,a)}{\hat{\pi}_0(s,a)}\left[R(s,a) - \hat{R}(s,a)\right]\right).
\end{align*}

One advantage of the doubly robust estimator is that it works if either $\hat{\pi}_0 = \pi_0$ or $\hat{R}(s,a) = R(s,a)$.

> i. [4 points] Please show that the doubly robust estimator is equal to $\mathbb{E}_{s\sim p(s)\\a \sim \pi_1(s,a)} R(s,a)$ when $\hat{\pi}_0 = \pi_0$.

> ii. [3 points] Please show that the doubly robust estimator is equal to $\mathbb{E}_{s\sim p(s)\\a \sim \pi_1(s,a)} R(s,a)$ when $\hat{R}(s,a) = R(s,a)$.

### Answer:

> i. If $\hat{\pi}_0 = \pi_0$, then 
\begin{align*}
\mathbb{E}_{s\sim p(s), a \sim \pi_0(s,a)} \left( \left[\mathbb{E}_{a\sim\pi_1(s,a)}\hat{R}(s,a)\right] + \frac{\pi_1(s,a)}{\hat{\pi}_0(s,a)}\left[R(s,a) - \hat{R}(s,a)\right]\right)
& = \mathbb{E}_{s\sim p(s), a \sim \pi_0(s,a)} \left[\mathbb{E}_{a\sim\pi_1(s,a)}\hat{R}(s,a)\right] + \mathbb{E}_{s\sim p(s), a \sim \pi_0(s,a)} \left(\frac{\pi_1(s,a)}{\hat{\pi}_0(s,a)}\left[R(s,a) - \hat{R}(s,a)\right]\right)\\ \\
& = \mathbb{E}_{s\sim p(s)} \left[\mathbb{E}_{a\sim\pi_1(s,a)}\hat{R}(s,a)\right] + 
\mathbb{E}_{s\sim p(s), a \sim \pi_1(s,a)} \left[R(s,a) - \hat{R}(s,a)\right]\\ \\
& = \mathbb{E}_{s\sim p(s), a\sim\pi_1(s,a)} \left[\hat{R}(s,a)\right] + 
\mathbb{E}_{s\sim p(s), a \sim \pi_1(s,a)} \left[R(s,a)\right] - \mathbb{E}_{s\sim p(s), a \sim \pi_1(s,a)} \left[\hat{R}(s,a)\right]\\ \\
& = \mathbb{E}_{s\sim p(s), a \sim \pi_1(s,a)} \left[R(s,a)\right].
\end{align*}

> ii. If $\hat{R}(s,a) = R(s,a)$, then 
\begin{align*}
\mathbb{E}_{s\sim p(s), a \sim \pi_0(s,a)} \left( \left[\mathbb{E}_{a\sim\pi_1(s,a)}\hat{R}(s,a)\right] + \frac{\pi_1(s,a)}{\hat{\pi}_0(s,a)}\left[R(s,a) - \hat{R}(s,a)\right]\right)
& = \mathbb{E}_{s\sim p(s), a \sim \pi_0(s,a)} \left[\mathbb{E}_{a\sim\pi_1(s,a)}\hat{R}(s,a)\right] + 0\\ \\
& = \mathbb{E}_{s\sim p(s)} \left[\mathbb{E}_{a\sim\pi_1(s,a)}\hat{R}(s,a)\right] \\ \\
& = \mathbb{E}_{s\sim p(s), a\sim\pi_1(s,a)} \left[\hat{R}(s,a)\right] \\ \\
& = \mathbb{E}_{s\sim p(s), a \sim \pi_1(s,a)} \left[R(s,a)\right].
\end{align*}

## (e) [2 points] 
We will now consider several situations where you might have a choice between the importance sampling estimator and the regression estimator. Please state whether the importance sampling estimator or the regression estimator would probably work best in each situation and explain why it would work better. In all of these situations, your states $s$ consist of patients, your actions a represent the drugs to give to certain patients and your $R(s,a)$ is the lifespan of the patient after receiving the drug.

> i. [1 points] Drugs are randomly assigned to patients, but the interaction between the drug, patient and lifespan is very complicated.

> ii. [1 points] Drugs are assigned to patients in a very complicated manner, but the interaction between the drug, patient and lifespan is very simple.

### Answer:

> i. In this case, the function $R(s,a)$ is assumed to be very complicated. Therefore, it seems that using importance sampling estimator is more convinsing since it does not need to estimate $R(s,a)$. It only needs to estimate $\pi_0$ which is assumed to be simple. 

> ii. In this case, the policy $\pi_0$ is assumed to be very complicated. Therefore, it seems that using regression estimator is more reliable  since it only needs to estimate $R(s,a)$ which is assumed to be simple.